#Task 1: Обучить полносвязную модель на MNIST

In [ ]:
import torch
from torch import nn
from torch import optim
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np

# Задаем трансформации для предобработки данных
transform = transforms.Compose([
    transforms.ToTensor(),  # Перевод картинки в тензор
    transforms.Normalize((0.5,), (0.5,))  # Нормализация
])

# Загрузка обучающего и тестового датасетов
train_data = datasets.MNIST('./data', download=True, train=True, transform=transform)
test_data = datasets.MNIST('./data', download=True, train=False, transform=transform)

# DataLoader'ы для итерации по датасетам
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)

# Определение модели
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(28*28, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)

    def forward(self, x):
        x = x.view(x.shape[0], -1)  # Выпрямляем вход в вектор
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

model = MLP()

# Определение функции потерь и оптимизатора
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.003)

# Обучение модели
epochs = 10
for e in range(epochs):
    running_loss = 0
    for images, labels in train_loader:
        # Обнуляем градиенты
        optimizer.zero_grad()
        
        # Прямое распространение
        output = model(images)
        loss = criterion(output, labels)
        
        # Обратное распространение и шаг оптимизации
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    else:
        print(f"Значения ф-ии потерь: {running_loss/len(train_loader)}")

# Тестирование модели
correct_count, all_count = 0, 0
for images, labels in test_loader:
    images = images.view(images.shape[0], -1)
    with torch.no_grad():
        logps = model(images)

    _, predicted = torch.max(logps, 1)
    correct_count += (predicted == labels).sum().item()
    all_count += labels.size(0)

print(f"Кол-во протестированных изображений = {all_count}")
print(f"\nТочность модели = {correct_count / all_count}")

# Визуализация результатов
dataiter = iter(test_loader)
images, labels = next(dataiter)
images = images[:5]
labels = labels[:5]

# Визуализируем изображения
def imshow(img):
    img = img * 0.5 + 0.5  # денормализация
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

imshow(torchvision.utils.make_grid(images))
# Истинные значения
print('Истинные: ', ' '.join('%5s' % labels[j].item() for j in range(5)))

# Предсказанные значения
images = images.view(images.shape[0], -1)
outputs = model(images)
_, predicted = torch.max(outputs, 1)

print('Предсказанные: ', ' '.join('%5s' % predicted[j].item() for j in range(5)))

Ядро каждый раз умирает при обработке. Добавлю файл в python, а также результаты:
Значения ф-ии потерь: 0.3382380364704996
Значения ф-ии потерь: 0.17378159143975866
Значения ф-ии потерь: 0.13243151394459707
Значения ф-ии потерь: 0.1215038654818408
Значения ф-ии потерь: 0.10883068691701817
Значения ф-ии потерь: 0.10141729995874423
Значения ф-ии потерь: 0.09516162840626649
Значения ф-ии потерь: 0.09144182974146976
Значения ф-ии потерь: 0.08367104561893202
Значения ф-ии потерь: 0.08125265750007382
Кол-во протестированных изображений = 10000

Точность модели = 0.9719
Истинные:      7     2     1     0     4
Предсказанные:      7     2     1     0     4